# Simple Moving Average Crossover Strategy

Let's start with a classic and simple trading strategy: the moving average crossover. The idea behind this strategy is to buy a stock when its fast simple moving average (SMA) crosses above its slow SMA and to short it when the fast SMA crosses below the slow SMA.

Let's start with our imports in the cell below, and preparing our data frame in the cell below that.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")


In [ ]:
url = "https://raw.githubusercontent.com/ImperialCollegeLondon/efds-ta-python/main/data/JPM_data.csv"
df = pd.read_csv(url)
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

## Compute and Plot SMAs

### Exercise 1

Calculate the fast (short-term) and slow (long-term) SMAs. For this example, use 20-day and 50-day SMAs.

In [ ]:
## YOUR CODE GOES HERE


Remember we'll have a window-length of NaNs in our data frame. This is because we don't have enough data points to calculate an N-day moving average until the Nth day. 
Let's plot the closing price with the SMAs to visually inspect the crossover points.

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(df["Close"], label='Close Price')
plt.plot(df["50-day SMA"], label="50-day SMA", color="blue")
plt.plot(df["200-day SMA"], label="200-day SMA", color="green")
plt.legend()
plt.title('Close Price with SMAs')
plt.show()

## The Crossover

To determine the crossover points, we will create a new column signal which will contain 1 for buy signals (when the fast SMA is above the slow SMA) and -1 for sell signals. Because we are in a *wait* position while we *wait* for the 200-day SMA to kick-in, we'll fill the Signal column with 0 for those rows. We'll use `where()` for both of these.

In [ ]:
df["Signal"] = np.where(df["50-day SMA"] > df["200-day SMA"], 1, -1)
df["Signal"] = np.where(df["200-day SMA"].isna(), 0, df["Signal"])
df

### Exercise 2

Can you identify on which dates our strategy would have prompted us to initiate a trade (i.e. buy or sell)?

Hint: the `diff()` function calculates the difference between rows for a column of the data frame. Can you use it to create a condition, then index the data frame using that condition?

In [ ]:
## YOUR CODE GOES HERE

## Backtesting

Backtesting is the process of testing a trading strategy on relevant historical data. We can calculate how it would have performed, had it been applied in the past. The simplest backtest calculates the returns that would have been realised if the strategy was implemented.

First, we create a new column with the signal from the previous day. When we backtest, we have to imagine we're trading in real time. Since our signals are based on close prices, we won't be able to act on them until the following day when the market re-opens. Failing to do this is called *look-ahead bias*.

In [ ]:
df['Position'] = df['Signal'].shift(1)
df

Next we calculate the market daily returns normally using `pct_change()`. We then calculate our strategy returns by multiplying our buy/sell position with the market returns. Here is how it works:

- On days when the market gains (+ market return)
    - In the buy position (+) , our strategy gains
    - In the sell position (-), our strategy loses
- On days when the market loses (- market return)
    - In the buy position (+), our strategy loses
    - In the sell position (-), our strategy gains

In [ ]:
# Calculating the daily market return
df["Market Return"] = df["Close"].pct_change()

# Calculating the strategy daily return
df["Strategy Return"] = df["Position"] * df["Market Return"]

df

To determine if our strategy would profit or lose, we can calculate the cumulative returns of our strategy for the period. We can then compare it to a *buy & hold* strategy (where we buy at the start of the period and sell at the end), which is represented as the cumulative market returns.

In [ ]:
df["Market Cumulative"] = (1 + df["Market Return"]).cumprod()
df["Strategy Cumulative"] = (1 + df["Strategy Return"]).cumprod()

df.tail()

Let's plot these for a visual representation.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df["Market Cumulative"], label="Market Returns")
plt.plot(df["Strategy Cumulative"], label="Strategy Returns")
plt.legend()
plt.title("Cumulative Returns of Market vs Strategy")
plt.show()